In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-bengali/classification


In [7]:
path = Path('./')

In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/soham-articles/bn/bn-train.csv', header=None)
df_train.head()

,0,1
0,entertainment,"প্রজাপতি প্রজাপতি আমার ইচ্ছে হয়ে, বনে বনে ঘাসে..."
1,state,দীর্ঘ সাত মাসের প্রতীক্ষার পর সোমবার সকালে এসে...
2,state,দিঘার হোটেল থেকে উদ্ধার হল যুবক-যুবতীর ঝুলন্ত ...
3,sports,কিছুদিন আগেই রাজ্যের গাইড বনে গিয়েছিলেন চেতেশ্...
4,national,মারুতির কারখানায় হামলা এবং ম্যানেজারকে পুড়িয়ে...


In [9]:
Counter(list(df_train[0]))

Counter({'entertainment': 1186,
         'international': 526,
         'kolkata': 4603,
         'national': 1435,
         'sports': 1289,
         'state': 2245})

In [10]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/soham-articles/bn/bn-valid.csv', header=None)
df_valid.head()

,0,1
0,entertainment,পোশাক বদলানোর মতো তিনি নাকি সঙ্গী বদলান। বি-টা...
1,sports,সৌরভ গঙ্গোপাধ্যায়ের সঙ্গে অ্যান্ড্রু ফ্লিনটফের...
2,kolkata,হোটেলের নিরাপত্তাকর্মীর ভুল বোঝাবুঝিতে অনুষ্ঠা...
3,kolkata,পরীক্ষার আগের সাজেশন হুবুহু মেলেনি শাসকের। লোক...
4,kolkata,১৪ জুলাই রেজ্জাক মোল্লার ইফতার পার্টিতে আমন্ত্...


In [11]:
df_test = pd.read_csv(path/'../../classification_public_datasets/soham-articles/bn/bn-test.csv', header=None)
df_test.head()

,0,1
0,kolkata,কেউ কেউ আজ ঘর ছেড়ে বেরোননি গোলমালের ভয়ে। কেউ ...
1,entertainment,সদ্য মাসি হয়েছেন। তাঁর বোনের ছেলে তৈমুরকে নিয়ে...
2,national,বেওয়ারিশ কুকুর দিয়ে হামলা হতে পারে দিল্লি বিমা...
3,kolkata,নিজস্ব প্রতিনিধি: মেয়র শোভন চট্টোপাধ্যায়ের বা...
4,national,খাতায় কলমে বিশ্বের বৃহত্তম গণতন্ত্র ভারতবর্ষ। ...


In [12]:
df_train.shape, df_valid.shape, df_test.shape

((11284, 2), (1411, 2), (1411, 2))

In [13]:
# from inltk.inltk import get_similar_sentences, tokenize
# import warnings
# warnings.filterwarnings("ignore")
# # aug_sen = []
# # aug_label = []
# for index, row in df_train[len(aug_sen)//2:].iterrows():
#     augs = get_similar_sentences(row[1], no_of_variations=2, language_code='bn', degree_of_aug=0.3)
#     aug_sen.extend(augs)
#     aug_label.extend([row[0]]*len(augs))
# df_aug = pd.DataFrame({0: aug_label, 1: aug_sen })
# df_aug.head()
# df_aug.to_csv('soham_articles_train_aug.csv', index=False)

In [14]:
df_train_aug = pd.read_csv('soham_articles_train_aug.csv')
df_train_aug.head()

,0,1
0,entertainment,"প্রজাপতি প্রজাপতি আপনার ইচ্ছে হয়ে, বনে বনে ঘা..."
1,entertainment,"প্রজাপতি কাঠঠোকরা আমার ইচ্ছে হয়ে, বনে বনে লাঙ..."
2,state,দীর্ঘ সাত মাসের প্রতীক্ষার পরে বুধবার ভোরে গিয...
3,state,দীর্ঘ সাত মাস তুখোড়ার পর সোমবার সকালে এসে পৌঁ...
4,state,দিঘার রিসর্ট থেকে উদ্ধার হল যুবক-ইউনিভার্সিটিত...


In [15]:
cutoff = int(0.01*len(df_train))
df_train = pd.DataFrame({0: list(df_train[0])[:cutoff] + list(df_train_aug['0'])[:2*cutoff], 1: list(df_train[1])[:cutoff] + list(df_train_aug['1'])[:2*cutoff]})
df_train.shape

(336, 2)

In [16]:
Counter(list(df_train[0]))

Counter({'entertainment': 39,
         'international': 18,
         'kolkata': 144,
         'national': 48,
         'sports': 24,
         'state': 63})

In [17]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [18]:
label_cols = [0]

In [19]:
class BengaliTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/bengali/tokenizer/bengali_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [20]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/bengali/tokenizer/bengali_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [21]:
# 30,000 is the vocab size that we chose in sentencepiece
bengali_vocab = Vocab(itos)

In [22]:
tokenizer = Tokenizer(tok_func=BengaliTokenizer, lang='bn')

In [23]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [24]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, label_cols=label_cols, tokenizer=tokenizer, vocab=bengali_vocab, bs=16)

In [25]:
data_lm.show_batch()

idx,text
0,ছেন ▁ । ▁যেকোন ও ▁দিনই ▁আসতে ▁পারে ▁সেই ▁ভয়ঙ্কর ▁দিনটি । ▁এখন ▁তাঁর ▁বড় ▁মেয়ে ▁অন্তর া ▁চৌধুরীর ▁কাছেই ▁আছেন ▁তিনি । ▁রুবি ▁হাসপাতালের ▁খুব ▁কাছেই ▁বড় ▁মেয়ের ▁কাছেই ▁কাটাতে ▁চান ▁জীবনের ▁শেষ ▁কয়েকটি ▁দিন । ▁আজ ই ▁এ বেলা . ইন ▁কে ▁জানালেন ▁অন্তর া । ▁স্বামী ▁সলিল ▁চৌধুরীর ▁অনুরোধে ই ▁প্রথম ▁অ্যালবামে ▁গাওয়া ▁গান ▁মরি ▁হায় ▁গো ▁হায় । ▁সব িতা র ▁কণ্ঠস্বর ের ▁জাদু তে ▁সেই
1,"▁কাউন্সিলর ▁দেব া শিস ▁মুখোপাধ্যায়ের ▁আশঙ্কা , ▁‘‘ বিরোধী ▁কাউন্সিলর দের ▁এলাকা ▁থেকে ▁১০০ ▁দিনের ▁সাফা ই কর্মী ▁তুলে ▁নেওয়া ▁হবে ▁না ▁তো ! ’’ ▁x x bo s ▁হরিয়ানা র ▁ছ ’ ▁বছর ▁বয়স ি ▁কৌ টি ল্য ▁পণ্ডিত কে ▁মনে ▁আছে ? ▁কিছু ▁কাল ▁আগে ▁এই ▁বিস্ময় বালক ▁নিজের ▁অবিশ্বাস্য ▁স্মৃতিশক্তি র ▁সুবাদে ▁সংবাদপত্রের ▁শিরোনাম ▁হয়েছিল । ▁২১ ৩টি ▁রাষ্ট্রের ▁যাবতীয় ▁গুরুত্বপূর্ণ ▁পরিসংখ্যান ▁একেবারে ▁ঠোঁট ের ▁গোড়ায় ▁ছিল"
2,"▁সঠিক ▁পরীক্ষা ▁না ▁করে ই ▁তাঁদের ▁ছেলেকে ▁ছেড়ে ▁দেওয়া ▁হয়েছে ▁বলে ▁অভিযোগ ▁তুলেছেন ▁শিশুর ▁বাবা - মা । ▁তাঁদের ▁মতো ▁অনেক ▁রোগীর ▁পরিবারের ই ▁প্রশ্ন , ▁‘‘ সব ▁ক্ষেত্রে ▁কেন ই ▁বা ▁এস ▁এস ▁কে ▁এম ের ▁উপর ▁নির্ভর ▁করতে ▁হবে ? ▁অন্যান্য ▁সরকারি ▁হাসপাতালে ▁কেন ▁সমস্যার ▁সুরা হা ▁হবে ▁না ? ’’ ▁বিষয়টি ▁নিয়ে ▁তদন্তের ▁আশ্বাস ▁দিয়েছেন ▁চিত্তরঞ্জন ের ▁অধ্যক্ষ ▁সু ত পা ▁গঙ্গোপাধ্যায় । ▁তিনি ▁বলেন , ▁‘‘"
3,"▁বা ▁বিদেশে র ▁মাটিতে ▁দাঁড়িয়ে ▁নিজের ▁দেশের ▁বিরোধী দের ▁সমালোচনা ▁না ▁করা ই ▁ দস্ত ুর । ▁তাই ▁আজ ▁কংগ্রেসের ▁জোরালো ▁প্রতিক্রিয়া । ▁বিস্মিত ▁পর্যবেক্ষক রাও । ▁তবে ▁তাঁরা ▁বলছেন , ▁বিতর্ক ▁থেকে ▁বাদ ▁যাননি ▁মনমোহন ▁সিংহ ও । ▁বিজেপি ▁পরমাণু ▁চুক্তিতে ▁আপত্তি ▁করায় ▁বিদেশে র ▁মাটিতে ▁মুখ ▁খুলে ▁তাঁদের ▁‘ অ বাক ▁পিছু টান ের ’ ▁সমালোচনা ▁করেছিলেন ▁মনমোহন । ▁তখন ▁প্রধানমন্ত্রীর ▁বিরুদ্ধে ▁রাজনৈতিক ▁ও ▁কূটনৈতিক ▁ শিষ্ট াচার"
4,▁মন্ত্রী । ▁ওয়েব ▁ডেস্ক : ▁হাসপাতাল ▁থেকে ▁ছাড়া ▁পেলেন ▁পরিবহণ ▁মন্ত্রী ▁মদন ▁মিত্র । ▁সকাল ▁১১ ▁টার ▁সময় ▁উড ল্যান্ড ▁হাসপাতাল ▁থেকে ▁ছাড়া ▁পেয়ে ▁সোজা ▁বাড়ির ▁দিকে ▁রওনা ▁হন । ▁তবে ▁আদালত ের ▁নির্দেশে ▁' গৃহ বন্দি ' ▁থাকবেন ▁মন্ত্রী । ▁গত ▁মঙ্গলবার ▁থেকে ▁' ফিট ' ▁মদন ▁হঠ াত ই ▁অসুস্থ ▁হলে ▁তাঁকে ▁ত ড়ি ঘড়ি ▁উড ল্যান্ড ▁হাসপাতালে ▁ভর্তি ▁করা ▁হয় । ▁হাসপাতালের ▁চিকিত্সক ▁স রোজ ▁মণ্ডল ▁জানিয়েছেন


In [26]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [27]:
# Loading the pretrained language model on bengali wikipedia
learn.load('../../../models/bengali/lm/ULMFiT/third_bn_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (336 items)
x: LMTextList
▁x x bo s ▁প্রজাপতি ▁প্রজাপতি ▁আমার ▁ইচ্ছে ▁হয়ে , ▁বনে ▁বনে ▁ঘাস ে ▁ঘাস ে ▁ও ড়ে ▁আর ▁ফেরে ... ▁সেই ▁সুর ▁যা ▁ফিরিয়ে ▁দেয় ▁শৈশব ের ▁সুগন্ধ । ▁সব িতা ▁চৌধুরী । ▁যাঁর ▁কণ্ঠ ▁অপূর্ব ▁সব ▁গান গুচ্ছ ▁উপহার ▁দিয়েছে ▁সব ▁প্রজন্মের ▁গান প্রেমী দের । ▁দুর ারোগ্য ▁কর্কট রোগ ে ▁আক্রান্ত ▁সেদিন ের ▁সব িতা । ▁গত ▁কয়েক ▁বছর ▁ধরে ▁নিরন্তর ▁লড়াইয়ের ▁পর ▁এখন ▁প্র হর ▁গুন ছেন ▁ । ▁যেকোন ও ▁দিনই ▁আসতে ▁পারে ▁সেই ▁ভয়ঙ্কর ▁দিনটি । ▁এখন ▁তাঁর ▁বড় ▁মেয়ে ▁অন্তর া ▁চৌধুরীর ▁কাছেই ▁আছেন ▁তিনি । ▁রুবি ▁হাসপাতালের ▁খুব ▁কাছেই ▁বড় ▁মেয়ের ▁কাছেই ▁কাটাতে ▁চান ▁জীবনের ▁শেষ ▁কয়েকটি ▁দিন । ▁আজ ই ▁এ বেলা . ইন ▁কে ▁জানালেন ▁অন্তর া । ▁স্বামী ▁সলিল ▁চৌধুরীর ▁অনুরোধে ই ▁প্রথম ▁অ্যালবামে ▁গাওয়া ▁গান ▁মরি ▁হায় ▁গো ▁হায় । ▁সব িতা র ▁কণ্ঠস্বর ের ▁জাদু তে ▁সেই ▁গান ▁শাশ্বত ▁হয়ে ▁আছে । ▁এখন ▁এই ▁আনন্দের ▁শহরে ▁এক লা ▁শুয়ে ▁সব িতা । ▁সুরের ▁সেই ▁ঝ র - ঝ র - ঝ র ণা ▁যে ▁আজ ▁নি রু চ্চা র,▁x x bo s ▁দীর্ঘ ▁সাত ▁মাসের ▁প্রতীক্ষ ার ▁পর 

In [28]:
learn.freeze()

In [29]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,6.070079,5.895398,0.182019,00:17


In [30]:
learn.unfreeze()

In [31]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.927559,5.926319,0.183334,00:18


In [32]:
learn.predict('সামগ্রিক পরিস্থিতি',n_words=10)

'সামগ্রিক পরিস্থিতি ▁ঘটনা ▁সিস্টেম ▁ছাগল ের ▁খেতে ▁সিল ▁চালানো ▁হল । ▁উত্তর'

In [33]:
learn.save_encoder('fine_tuned_enc')

In [34]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=bengali_vocab, label_cols=label_cols, bs=16)

In [35]:
data_clas.show_batch()

text,target
"▁x x bo s ▁পদস্থ ▁অফিসার ▁থেকেই ▁থানার ▁এবং সি , ▁বাহিনী ▁কাছে ▁ছোটখাট ▁‘ ব্রি ফ িং ' ▁ছিলো ▁তাঁর — ▁' তোমরা ▁বর্তমানে ▁নির্বাচন ▁কমিশনের ▁অধীন ।"" ▁ফলে ▁ফেডারেশনের ▁গাইড লেজ ▁এবং ▁সাধারণ ▁সকল ▁নিয়ম ▁মেনে ▁চলতে ▁হবে । ▁কাউকে ▁কোনও ▁ভাবে ▁রে য়াত ▁করার ▁প্রয়োজন ▁নেই ।"" ▁কোনও ▁অনুরোধ - উপ রোধ ে ▁কান ▁দেওয়ার ▁প্রয়োজন ▁আছে । ▁আমরা ▁এরকম কিছু ▁করবো ▁না , ▁যেখানে ▁আমাদের ▁প্রতিষ্ঠানের",state
"▁x x bo s ▁পদস্থ ▁অফিসার ▁থেকে ▁থানার ▁ও সি , ▁বাহিনীর ▁কাছে ▁সংক্ষিপ্ত ▁‘ ব্রি ফ িং ’ ▁ছিল ▁তাঁর — ▁‘ আমরা ▁এখন ▁নির্বাচন ▁কমিশনের ▁অধীন । ▁ফলে ▁কমিশনের ▁গাইড বুক ▁এবং ▁সাধারণ ▁সমস্ত ▁নিয়ম ▁মেনে ▁চলতে ▁হবে । ▁কাউকে ▁কোনও ভাবে ▁রে য়াত ▁করার ▁দরকার ▁নেই । ▁কোনও ▁অনুরোধ - উপ রোধ ে ▁কান ▁দেওয়ার ▁প্রয়োজন ▁নেই । ▁আমরা ▁এমন কিছু ▁করব ▁না , ▁যাতে ▁আমাদের ▁প্রতিষ্ঠানের",state
"▁x x bo s ▁পদস্থ ▁অফিসার ▁থেকে ▁থানার ▁ও সি ▁এবং ▁বাহিনীর ▁নিকটে ▁সংক্ষিপ্ত ▁“ ব্রি ংক িং ’’ ▁ছিল ▁তাঁর ▁– ▁‘ আমরা ▁এখনো ▁নির্বাচনে ▁কমিশনের ▁অধীন । ▁ফলে ▁কমিশনের ▁গাইড বুক ▁ও ▁সাধারণ ▁সমস্ত ▁নিয়ম ▁মেনে ▁চললে ▁হবে । ▁কাউকে ▁কোনও ভাবে ▁রে য়াত ▁করার ▁দরকার ▁নেই । ▁কোন ▁অনুরোধ - শুধু রোধ ে ▁কান ▁দেওয়ার ▁প্রয়োজনীয়তা ▁নাই । ▁আমি ▁একধরনে বিভিন্ন ▁করব ▁নাই , ▁যাতে ▁আমাদের ▁সংগঠনের",state
▁x x bo s ▁রাজপথ ই ▁ইতিহাস ▁তৈরি ▁করে । ▁আন্দোলন ▁কিংবা ▁ক্ষমতার । ▁রেড ▁রোড ▁সাক্ষী ▁থাকল ▁মমতা ▁বন্দ্যোপাধ্যায় ের ▁দ্বিতীয় ▁ইনিংসে র ▁আনুষ্ঠানিক ▁প্রতিষ্ঠার । ▁তবে ▁এই ▁রাজপথ ে ▁রাজ সূ য়ের ▁বহু ▁শতক ▁আগে ▁থেকেই ▁রেড ▁রোড ▁নানা ▁ইতিহাসের ▁সাক্ষী । ▁২০১০ ▁সালের ▁পুর ভো টে ▁তৃণমূল ের ▁প্রতিশ্রুতি র ▁ঢ ের ▁আগে ▁ইংরেজ ▁কোম্পানি ▁কলকাতা ▁শহরকে ▁লন্ডনের ▁ধাঁচে ▁গড়ে ▁তুলতে ▁চেয়েছিল । ▁শুধু ▁সরকারি ▁কাগজপত্র ে,state
▁x x bo s ▁দূরপাল্লার ই ▁ইতিহাসে ▁তৈরি ▁করে৷ ▁আন্দোলন ▁কিংবা ▁ক্ষমতার ৷ ▁রেড ▁রোডে ▁সাক্ষী ▁থাকল ▁অবনীন্দ্রনাথ ▁বন্দ্যোপাধ্যায় ের ▁দ্বিতীয় ▁ইনিংসে র ▁আনুষ্ঠানিক ▁প্রতিষ্ঠার । ▁তবে ▁এই ▁রাজপথ েই ▁রাজ সূ য়ের ▁বহু ▁শতক ▁আগে ▁থেকেই ▁রেড ▁রোড ▁নানা ▁ইতিহাস ▁মুখাবয়ব । ▁২০১০ ▁সালের ▁পুর হেনরি ট ▁তৃণমূল েরা ▁প্রতিশ্রুতি ের ▁ঢ ের ▁আগে ▁ইংরেজ ▁কোম্পানি ▁কোলকাতা ▁শহরটিতে ▁মুম্বাইয়ের ▁ধাঁচে ▁গড়ে ▁তুলছে ▁চাননি । ▁শুধু ▁সরকারি ▁সাইনবোর্ড ে ▁“,state


In [36]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [37]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [38]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (336 items)
x: TextList
▁x x bo s ▁প্রজাপতি ▁প্রজাপতি ▁আমার ▁ইচ্ছে ▁হয়ে , ▁বনে ▁বনে ▁ঘাস ে ▁ঘাস ে ▁ও ড়ে ▁আর ▁ফেরে ... ▁সেই ▁সুর ▁যা ▁ফিরিয়ে ▁দেয় ▁শৈশব ের ▁সুগন্ধ । ▁সব িতা ▁চৌধুরী । ▁যাঁর ▁কণ্ঠ ▁অপূর্ব ▁সব ▁গান গুচ্ছ ▁উপহার ▁দিয়েছে ▁সব ▁প্রজন্মের ▁গান প্রেমী দের । ▁দুর ারোগ্য ▁কর্কট রোগ ে ▁আক্রান্ত ▁সেদিন ের ▁সব িতা । ▁গত ▁কয়েক ▁বছর ▁ধরে ▁নিরন্তর ▁লড়াইয়ের ▁পর ▁এখন ▁প্র হর ▁গুন ছেন ▁ । ▁যেকোন ও ▁দিনই ▁আসতে ▁পারে ▁সেই ▁ভয়ঙ্কর ▁দিনটি । ▁এখন ▁তাঁর ▁বড় ▁মেয়ে ▁অন্তর া ▁চৌধুরীর ▁কাছেই ▁আছেন ▁তিনি । ▁রুবি ▁হাসপাতালের ▁খুব ▁কাছেই ▁বড় ▁মেয়ের ▁কাছেই ▁কাটাতে ▁চান ▁জীবনের ▁শেষ ▁কয়েকটি ▁দিন । ▁আজ ই ▁এ বেলা . ইন ▁কে ▁জানালেন ▁অন্তর া । ▁স্বামী ▁সলিল ▁চৌধুরীর ▁অনুরোধে ই ▁প্রথম ▁অ্যালবামে ▁গাওয়া ▁গান ▁মরি ▁হায় ▁গো ▁হায় । ▁সব িতা র ▁কণ্ঠস্বর ের ▁জাদু তে ▁সেই ▁গান ▁শাশ্বত ▁হয়ে ▁আছে । ▁এখন ▁এই ▁আনন্দের ▁শহরে ▁এক লা ▁শুয়ে ▁সব িতা । ▁সুরের ▁সেই ▁ঝ র - ঝ র - ঝ র ণা ▁যে ▁আজ ▁নি রু চ্চা র,▁x x bo s ▁দীর্ঘ ▁সাত ▁মাসের ▁প্রতীক্ষ ার ▁পর ▁সোমব

In [39]:
learn.freeze()

In [40]:
learn.loss_func.func

CrossEntropyLoss()

In [41]:
mcc = MatthewsCorreff()

In [42]:
learn.metrics = [mcc, accuracy]

In [43]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.476684,1.458465,0.509208,0.600284,00:13


In [44]:
learn.freeze_to(-2)
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.121589,1.023469,0.522776,0.649185,00:14
1,0.930453,0.844730,0.589055,0.701630,00:14
2,0.686908,0.850842,0.599929,0.706591,00:14


In [45]:
learn.save('second-full')

In [46]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (336 items)
x: TextList
▁x x bo s ▁প্রজাপতি ▁প্রজাপতি ▁আমার ▁ইচ্ছে ▁হয়ে , ▁বনে ▁বনে ▁ঘাস ে ▁ঘাস ে ▁ও ড়ে ▁আর ▁ফেরে ... ▁সেই ▁সুর ▁যা ▁ফিরিয়ে ▁দেয় ▁শৈশব ের ▁সুগন্ধ । ▁সব িতা ▁চৌধুরী । ▁যাঁর ▁কণ্ঠ ▁অপূর্ব ▁সব ▁গান গুচ্ছ ▁উপহার ▁দিয়েছে ▁সব ▁প্রজন্মের ▁গান প্রেমী দের । ▁দুর ারোগ্য ▁কর্কট রোগ ে ▁আক্রান্ত ▁সেদিন ের ▁সব িতা । ▁গত ▁কয়েক ▁বছর ▁ধরে ▁নিরন্তর ▁লড়াইয়ের ▁পর ▁এখন ▁প্র হর ▁গুন ছেন ▁ । ▁যেকোন ও ▁দিনই ▁আসতে ▁পারে ▁সেই ▁ভয়ঙ্কর ▁দিনটি । ▁এখন ▁তাঁর ▁বড় ▁মেয়ে ▁অন্তর া ▁চৌধুরীর ▁কাছেই ▁আছেন ▁তিনি । ▁রুবি ▁হাসপাতালের ▁খুব ▁কাছেই ▁বড় ▁মেয়ের ▁কাছেই ▁কাটাতে ▁চান ▁জীবনের ▁শেষ ▁কয়েকটি ▁দিন । ▁আজ ই ▁এ বেলা . ইন ▁কে ▁জানালেন ▁অন্তর া । ▁স্বামী ▁সলিল ▁চৌধুরীর ▁অনুরোধে ই ▁প্রথম ▁অ্যালবামে ▁গাওয়া ▁গান ▁মরি ▁হায় ▁গো ▁হায় । ▁সব িতা র ▁কণ্ঠস্বর ের ▁জাদু তে ▁সেই ▁গান ▁শাশ্বত ▁হয়ে ▁আছে । ▁এখন ▁এই ▁আনন্দের ▁শহরে ▁এক লা ▁শুয়ে ▁সব িতা । ▁সুরের ▁সেই ▁ঝ র - ঝ র - ঝ র ণা ▁যে ▁আজ ▁নি রু চ্চা র,▁x x bo s ▁দীর্ঘ ▁সাত ▁মাসের ▁প্রতীক্ষ ার ▁পর ▁সোমব

In [47]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.376727,0.812124,0.623319,0.725018,00:18
1,0.338108,0.799356,0.637152,0.733522,00:18
2,0.297844,0.779938,0.643715,0.739192,00:18
3,0.259544,0.772002,0.622924,0.725018,00:18
4,0.233743,0.776539,0.639286,0.734231,00:18


Better model found at epoch 0 with accuracy value: 0.7250177264213562.
Better model found at epoch 1 with accuracy value: 0.7335222959518433.
Better model found at epoch 2 with accuracy value: 0.7391920685768127.


In [48]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (336 items)
x: TextList
▁x x bo s ▁প্রজাপতি ▁প্রজাপতি ▁আমার ▁ইচ্ছে ▁হয়ে , ▁বনে ▁বনে ▁ঘাস ে ▁ঘাস ে ▁ও ড়ে ▁আর ▁ফেরে ... ▁সেই ▁সুর ▁যা ▁ফিরিয়ে ▁দেয় ▁শৈশব ের ▁সুগন্ধ । ▁সব িতা ▁চৌধুরী । ▁যাঁর ▁কণ্ঠ ▁অপূর্ব ▁সব ▁গান গুচ্ছ ▁উপহার ▁দিয়েছে ▁সব ▁প্রজন্মের ▁গান প্রেমী দের । ▁দুর ারোগ্য ▁কর্কট রোগ ে ▁আক্রান্ত ▁সেদিন ের ▁সব িতা । ▁গত ▁কয়েক ▁বছর ▁ধরে ▁নিরন্তর ▁লড়াইয়ের ▁পর ▁এখন ▁প্র হর ▁গুন ছেন ▁ । ▁যেকোন ও ▁দিনই ▁আসতে ▁পারে ▁সেই ▁ভয়ঙ্কর ▁দিনটি । ▁এখন ▁তাঁর ▁বড় ▁মেয়ে ▁অন্তর া ▁চৌধুরীর ▁কাছেই ▁আছেন ▁তিনি । ▁রুবি ▁হাসপাতালের ▁খুব ▁কাছেই ▁বড় ▁মেয়ের ▁কাছেই ▁কাটাতে ▁চান ▁জীবনের ▁শেষ ▁কয়েকটি ▁দিন । ▁আজ ই ▁এ বেলা . ইন ▁কে ▁জানালেন ▁অন্তর া । ▁স্বামী ▁সলিল ▁চৌধুরীর ▁অনুরোধে ই ▁প্রথম ▁অ্যালবামে ▁গাওয়া ▁গান ▁মরি ▁হায় ▁গো ▁হায় । ▁সব িতা র ▁কণ্ঠস্বর ের ▁জাদু তে ▁সেই ▁গান ▁শাশ্বত ▁হয়ে ▁আছে । ▁এখন ▁এই ▁আনন্দের ▁শহরে ▁এক লা ▁শুয়ে ▁সব িতা । ▁সুরের ▁সেই ▁ঝ র - ঝ র - ঝ র ণা ▁যে ▁আজ ▁নি রু চ্চা র,▁x x bo s ▁দীর্ঘ ▁সাত ▁মাসের ▁প্রতীক্ষ ার ▁পর ▁সোমব

In [49]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,state,entertainment,kolkata,national,international,sports
0,কেউ কেউ আজ ঘর ছেড়ে বেরোননি গোলমালের ভয়ে। কেউ ...,kolkata,kolkata,0.0659018,0.00883753,0.891875,0.0216975,0.00878853,0.00289982
1,সদ্য মাসি হয়েছেন। তাঁর বোনের ছেলে তৈমুরকে নিয়ে...,entertainment,entertainment,0.0522013,0.514537,0.241158,0.0942263,0.0653673,0.0325099
2,বেওয়ারিশ কুকুর দিয়ে হামলা হতে পারে দিল্লি বিমা...,national,kolkata,0.124577,0.00775788,0.543016,0.267967,0.0254723,0.0312101
3,নিজস্ব প্রতিনিধি: মেয়র শোভন চট্টোপাধ্যায়ের বা...,kolkata,kolkata,0.197221,0.00336302,0.767134,0.0262229,0.00443765,0.00162153
4,খাতায় কলমে বিশ্বের বৃহত্তম গণতন্ত্র ভারতবর্ষ। ...,national,entertainment,0.00583218,0.582562,0.0128055,0.0111146,0.0547542,0.332932


In [50]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7406094968107725

In [51]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.6508309934610141

In [52]:
df_result.to_csv('soham_articles_result_with_aug.csv', index=False)